## test sending emails

1. all addresses must be string objects separated with commas
2. if passed an invalid email address, the error message will be sent to the from addr

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib, ssl

import pandas as pd 
import numpy as np
import os

In [ ]:
msg = MIMEMultipart('alternative')
msg['Subject'] = 'test'
msg.attach(MIMEText('<p>this is a test run</p>', 'html'))

# all msg addresses expect a string object separated with commas
msg['From'] = os.getenv('REGLAB_TEST_GMAIL_ADDR')
# if passed an invalid email address, the error message will be sent to the from addr
msg['To'] = 'nlin@law.stanford.edu, nicole.hjlin@gmail.com, sdklfrankl238@gmail.com'
msg['Bcc'] = 'reglabtest@gmail.com'
password = os.getenv('REGLAB_TEST_GMAIL_PWD')
port = 465  # For SSL
smtp_server = "smtp.gmail.com"
context = ssl.create_default_context()

with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(msg['From'], password)
    server.send_message(msg)

## testing email sending systems

In [ ]:
system = 'sherlock'
assert system in ['sherlock', 'whippet'], 'Expect system to be in sherlock or whippet. Aborting.'

## copy prod folders to test folder for testing on whippet 

In [ ]:
import os
import subprocess
import boto3

## S3 bucket Connection
BUCKET_NAME = 'epa-esnc-notif'
BUCKET_ACCESS_ID = os.getenv('REGLAB_AWS_ACCESS_ID')
BUCKET_ACCESS_KEY = os.getenv('REGLAB_AWS_ACCESS_KEY')
S3_RESOURCE = boto3.resource(service_name='s3')
BUCKET = S3_RESOURCE.Bucket(BUCKET_NAME)
## S3 bucket
S3_PROJECT_DIR = os.path.join('s3://', BUCKET_NAME)

In [ ]:
s3_project_dir = S3_PROJECT_DIR 
run_id = '2021Q4_2021-08-04_140828_785316'
os.system(f'aws s3 cp {s3_project_dir}/prod/ {s3_project_dir}/test/ --recursive')

## generating ky batch report as pdf 

In [ ]:
# first set working directory as where the 0_email_maker will sit
import os 

os.chdir('..')
os.getcwd()

In [ ]:
import os
import pandas as pd
# import supporting modules
import configs

In [ ]:
mode = 'prod'
run_id = '2021Q4_2021-08-04_140828_785316'
oak_project_dir = configs.GET_OAK_PROJECT_DIR()
oak_run_dir = os.path.join(oak_project_dir, mode, run_id)

In [ ]:
ky_report = pd.read_csv(os.path.join(oak_run_dir, 'batch_report_ky.csv'))
ky_report.head()

In [ ]:
ky_report.iloc[0]

In [ ]:
import html2text
from IPython.core.display import display, HTML
display(HTML(ky_report.iloc[0]['body']))

In [ ]:
import fpdf 

pdf = fpdf.FPDF(format = 'letter')
pdf.add_page()
pdf.set_font('Arial', size = 12)

i = 0 
email = ky_report.iloc[i]
pdf.write("npdes_permit_id: ")
pdf.ln()
pdf.output('test.pdf')

In [ ]:
ky_report.iloc[0]

In [ ]:
ky_report.to_html()

## KY correction 2021 Q4 
manually select a pollutant to display for permits notified with no violations in the past quarter

In [ ]:
# first set working directory as where the 0_email_maker will sit
import os 

os.chdir('..')
os.getcwd()

In [ ]:
import os
import pandas as pd
import sqlalchemy as sa
import datetime as dt
import configs

In [ ]:
engine = configs.GET_ENGINE()

In [ ]:
query = """
SELECT
  npdes_permit_id,
  known_violations
FROM
  sandbox.esnc_notif_at_risk_permits
WHERE
  permit_state = 'KY'
  AND notification_flag
  AND NOT sample_violation_flag
"""
with engine.begin() as conn:
    df = pd.read_sql(query, conn)
df

In [ ]:
for i in range(len(df)):    
    viol_list = eval(df.known_violations[i])
    viol_df = pd.DataFrame(viol_list)
    viol_df = viol_df.sort_values(by = 'monitoring_period_end_date', ascending = False)
    print(viol_df.iloc[0]['monitoring_period_end_date'])

## Generate emails as PDFs for KY
- read in batch report
- compile emails in html format 
- print out pdfs

In [ ]:
# first set working directory as where the 0_email_maker will sit
import os 

os.chdir('..')
os.getcwd()

In [ ]:
# import installed packages
import os
import pdfkit
import numpy as np
import pandas as pd
import configs

In [ ]:
def generate_email_pdfs(run_id, mode, batch_report_file_name):
    oak_project_dir = configs.GET_OAK_PROJECT_DIR()
    oak_run_dir = os.path.join(oak_project_dir, mode, run_id)
    batch_report = pd.read_csv(os.path.join(oak_run_dir, batch_report_file_name))
    batch_report = batch_report.sort_values(by = 'npdes_permit_id')
    html_str = ""
    for i in range(len(batch_report)):
        email = batch_report.iloc[i]
        html_str_i = f"""
        <meta http-equiv="Content-type" content="text/html; charset=UTF-8" />
        <p style="page-break-before: always;"></p>
        <p><b>Subject:</b> {email['subject']}</p>
        <p><b>From:</b> {email['email_sent_from_addr']}</p>
        <p><b>To:</b> {email['email_sent_to_addr']}</p>
        <p><b>Timestamp:</b> {email['email_sent_timestamp']} EST</p>
        <hr>
        {email['body']}
        """
        html_str = html_str + html_str_i
    pdfkit.from_string(html_str, os.path.join(oak_run_dir, batch_report_file_name.replace('.csv', '.pdf')))

In [ ]:
generate_email_pdfs(run_id = '2021Q4_2021-08-04_140828_785316', mode = 'prod', batch_report_file_name = 'batch_report_ky.csv')

In [ ]:
generate_email_pdfs(run_id = 'ky_correction_2021Q4_2021-08-06_130754_810831', mode = 'prod', batch_report_file_name = 'batch_report_ky.csv')

### Generate emails as pdfs for TN

In [ ]:
# first set working directory as where the 0_email_maker will sit
import os 

os.chdir('..')
os.getcwd()

In [ ]:
# import installed packages
import os
import pdfkit
import numpy as np
import pandas as pd

In [ ]:
mode = 'prod'
run_id = '2021Q4_2021-08-04_140828_785316'
batch_report_file_name = 'batch_report_tn-2021_08_04.csv'

In [ ]:
oak_project_dir = '../esnc_risk_notif_processing/'
oak_run_dir = os.path.join(oak_project_dir, mode, run_id)
batch_report = pd.read_csv(os.path.join(oak_run_dir, batch_report_file_name))
batch_report = batch_report.sort_values(by = 'npdes_permit_id')
html_str = ""
for i in range(len(batch_report)):
    email = batch_report.iloc[i]
    html_str_i = f"""
    <meta http-equiv="Content-type" content="text/html; charset=UTF-8" />
    <p style="page-break-before: always;"></p>
    <p><b>Subject:</b> {email['subject']}</p>
    <p><b>From:</b> {email['email_sent_from_addr']}</p>
    <p><b>To:</b> {email['email_sent_to_addr']}</p>
    <p><b>Timestamp:</b> {email['email_sent_timestamp']} EST</p>
    <hr>
    {email['body']}
    """
    html_str = html_str + html_str_i
pdfkit.from_string(html_str, os.path.join(oak_run_dir, batch_report_file_name.replace('.csv', '.pdf')))